<a href="https://colab.research.google.com/github/nullpitch-dev/DS_L1_Notebooks/blob/master/DS_L1_EX_07_2nd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

url = 'https://raw.githubusercontent.com/nullpitch-dev/hj_public/master/R_pop_stat.csv'
data = pd.read_csv(url)

In [0]:
# [0] get_dummies

# drop NAs
base = data.dropna(subset=['AGE', 'INCOME', 'XGRP'])

# filter by age
base = base[(base['AGE'] >= 20) & (base['AGE'] <= 50)]

# get dummy variables
base = pd.get_dummies(base, columns=['SEX', 'SCHL', 'WORK'], drop_first=True)

In [37]:
# [1]

mean = base.INCOME.mean()
std = base.INCOME.std()

ul = mean + 3 * std
ll = mean - 3 * std

# check outliers
base = base.assign(outlier=base.INCOME.apply(lambda x: 1 if (x > ul) | (x < ll)
                                                         else 0))

# get max and min
max = base[base.outlier == 1].INCOME.max()
min = base[base.outlier == 1].INCOME.min()

print(f'Answer [1] : {max - min:.0f}')

base = base[base.outlier == 0]

Answer [1] : 3325000


In [38]:
# [2]

import math

corr = base[base.INCOME >= base.INCOME.quantile(0.75)][['INCOME', 'AGE']].corr(
                                                              method='spearman')
print(f'Answer [2] : {math.floor(corr.iloc[0, 1] * 1000) / 1000}')

Answer [2] : 0.115


In [39]:
# [3] chi2_contingency, pivot_table

from scipy.stats import chi2_contingency

# check if INCOME >= 25000
base = base.assign(h_income=base.INCOME.apply(lambda x: 'High' if x >=25000
                                                               else 'Low'))

# get original WORK column
data3 = pd.merge(base, data.WORK, how='left', left_index=True, right_index=True)

# make pivot talbe
pivot = data3.pivot_table(index=['WORK'], columns=['h_income'], aggfunc='size')

# perform chi2 test and get P-value
_, p_val, _, _ = chi2_contingency(pivot)

# decision 
if p_val < 0.05:
    decision = 'Reject H0, there is relationship'
else:
    decision = 'Cannot reject H0, there is no relationship'

print(f'Answer [3] : {p_val:.5f}, {decision}')

Answer [3] : 0.00044, Reject H0, there is relationship


In [40]:
# [4] LinearRegression, log10(x), 10 ** x

import math
from sklearn.linear_model import LinearRegression

# prepare train set
X_cols = list(base.columns[0:1]) + list(base.columns[3:-2])
train_X = base[base.XGRP <= 7][X_cols]
train_y = base[base.XGRP <= 7].INCOME.apply(lambda x: math.log10(x))

# train
lr = LinearRegression()
model = lr.fit(train_X, train_y)

# predict
sample = pd.DataFrame([[38, 0, 0, 0, 0, 0, 1, 0, 1, 0]], columns=X_cols)
pred = model.predict(sample)

print(f'Answer [4] : {round(10 ** pred[0], -2):.0f}')

Answer [4] : 47400


In [41]:
# [5] mean_squared_error

from sklearn.metrics import mean_squared_error
import numpy as np

# prepare test set
test_X = base[base.XGRP > 7][X_cols]
test_y = base[base.XGRP > 7].INCOME.apply(lambda x: math.log10(x))

# predict
pred_5 = model.predict(test_X)

# calculate root mean squared error
rmse = np.sqrt(mean_squared_error(test_y, pred_5))

print(f'Answer [5] : {rmse:.2f}')

Answer [5] : 0.29
